# Statement Extractor
> Statement extraction through claim detection

This Notebook guides you through the *Statement Extractor* module.

In [1]:
import pandas as pd

from src.utils import pdf_parser
from src import segmenter, detect_claims, highlight_pdf
from config import SAMPLE_PDF_PATH, MODEL_NAME, MODEL_WEIGHTS_PATH, SPACY_DATA_PATH

## 0. Load data

In [4]:
text = pdf_parser.pdf_to_text(SAMPLE_PDF_PATH)  # load all text from pdf

In [5]:
parse = pdf_parser.parse_text(text)  # parse pdf with specific document parser

## 1. Create segments

In [6]:
title = parse["title"]
date = parse["date"]
passages = parse["passages"]  # this document has natural passages

In [7]:
doc = []  # create empty doc

In [8]:
passage_id = 0
make_segments = False
for passage in passages:
    if make_segments:
        segments = segmenter.split_segments(passage.get("text"), segment_len=2)
    else:
        sentences = segmenter.split_sentences(passage.get("text"))
        segments = [{"sentence": sentence} for sentence in sentences]
    
    for segment in segments:
        segment["passage_id"] = passage_id
        doc.append(segment)
    
    passage_id += 1

### 1.1 Load to table

In [9]:
doc_table = pd.DataFrame(doc)

In [10]:
doc_table.head()

,sentence,passage_id
0,Guten Mittag liebe Journalistinnen und Journal...,0
1,Herzlich willkommen hier zu unserem virtuellen...,0
2,"Mein Name ist Bastian Zimmermann, und ich bin ...",0
3,"Ich freue mich, heute auch unsere drei Experte...",0
4,"Man hört immer wieder, wie wichtig Quantentech...",0


### 1.2 Add Metadata

In [11]:
doc_table["title"] = title
doc_table["date"] = date

In [12]:
doc_table["speaker"] = None
doc_table["timestamp"] = None

for idx, passage in enumerate(passages):
    speaker = passage.get("speaker")
    timestamp = passage.get("timestamp")
    
    doc_table['speaker'].loc[doc_table['passage_id'] == idx] = speaker
    doc_table['timestamp'].loc[doc_table['passage_id'] == idx] = timestamp


/home/jueri/.local/share/virtualenvs/statement_extractor-EMjVysve/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)



In [13]:
doc_table.head()

,sentence,passage_id,title,date,speaker,timestamp
0,Guten Mittag liebe Journalistinnen und Journal...,0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]
1,Herzlich willkommen hier zu unserem virtuellen...,0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]
2,"Mein Name ist Bastian Zimmermann, und ich bin ...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]
3,"Ich freue mich, heute auch unsere drei Experte...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]
4,"Man hört immer wieder, wie wichtig Quantentech...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]


## 2. Detect main concept
Topic Sentence similarity by sentence embedding (dot product of word embeddings)

In [32]:
from src.main_concept import sentence_title_similarity, wikified_article_score, wikify
from config import TAGME_TOKEN, DANDELION_TOKEN

In [ ]:
similarities = sentence_title_similarity(text, title, doc_table["sentence"].to_list())

In [49]:
doc_table["title_sentence_similaritie"] = similarities

Score as the sum of probabilitys from all spots in the title/introduction and sentence

In [58]:
introduction = """Die Omikron-Variante breitet sich in Deutschland weiter aus. Bundesgesundheitsminister Karl Lauterbach rechnet damit, dass die Variante hierzulande bald vorherrschend sein wird. Bei einer einfachen Fortschreibung der Wachstumsraten vor Weihnachten wäre dies schon jetzt der Fall. Aktuelle Studien deuten darauf hin, dass sich Omikron zwar rascher ausbreitet als vorherige Varianten, aber im Vergleich zur Delta-Variante zu milderen Krankheitsverläufen führt. So ist etwa die Viruslast in der Lunge niedriger als bei einer Delta-Infektion [I]. Krankenhausdaten aus Großbritannien stützen diese Beobachtungen: Während die Zahl der Corona-Patienten in den Kliniken steigt, stagniert die Zahl der genutzten Beatmungsplätze weitgehend [II]. Die Patienten werden also im Krankenhaus behandelt, benötigen aber offenbar seltener Atemhilfe. In Deutschland sinkt die Hospitalisierungsrate derzeit noch, allerdings ist anzunehmen, dass Omikron trotz steigender Impfquote aufgrund der hohen Infektiosität noch zu einer größeren Belastung für das Gesundheitssystem und die Wirtschaft werden könnte, als es die Daten bisher suggerieren. Bund und Länder wollen deshalb am Freitag unter anderem über etwaige neue Kontaktbeschränkungen beraten und darüber, ob die Quarantäneregeln angepasst werden. Denn mit Ausbreitung der Omikron-Variante wächst die Gefahr, dass auch die Zahl von Quarantäne-Anordnungen bei Kontaktpersonen von Infizierten steigt. Das Robert Koch-Institut empfiehlt derzeit im Fall von Omikron immer eine Quarantäne – auch für geimpfte und genesene Kontaktpersonen. Über die neuesten Erkenntnisse zur Omikron-Variante, den Immunschutz nach Impfung und die Lage in den Kliniken diskutierten Fachleute in einem 50-minütigen Press Briefing des SMC."""

In [66]:
title_wikifyed = wikify(text=title, service="tagme", token=TAGME_TOKEN)
intor_wikifyed = wikify(text=introduction, service="tagme", token=TAGME_TOKEN)


In [67]:
title_article_ids = {article["id"]for article in title_wikifyed}  # get all article ids from title
intro_article_ids = {article["id"]for article in intor_wikifyed}  # get all article ids from introduction

In [70]:
doc_table["title_wiki_score"] = doc_table.apply(lambda x: wikified_article_score(x["sentence"], title_article_ids), axis=1)

In [71]:
doc_table["introduction_wiki_score"] = doc_table.apply(lambda x: wikified_article_score(x["sentence"], intro_article_ids), axis=1)

In [78]:
doc_table.sort_values(by="introduction_wiki_score", ascending=False).head()

,segment_id,sentence,passage_id,title,date,speaker,timestamp,wiki_score,title_wiki_score,introduction_wiki_score
12,2,"Allerdings ist anzunehmen, dass aufgrund allei...",0,„Neue Erkenntnisse zu Omikron und Ausblick“,05.01.2022,Moderator [00:00:00],,0.583333,0.583333,1.932396
8,2,So scheint die Viruslast in der Lunge etwas ni...,0,„Neue Erkenntnisse zu Omikron und Ausblick“,05.01.2022,Moderator [00:00:00],,0.000000,0.000000,1.686103
5,2,Bundesgesundheitsminister Karl Lauterbach rech...,0,„Neue Erkenntnisse zu Omikron und Ausblick“,05.01.2022,Moderator [00:00:00],,0.000000,0.000000,1.489703
53,0,"Es gibt diese Studien, die suggerieren, dass d...",6,„Neue Erkenntnisse zu Omikron und Ausblick“,05.01.2022,Moderator [00:06:34],,0.000000,0.000000,1.295855
104,6,"Das heißt also, die Effekte in Großbritannien,...",11,„Neue Erkenntnisse zu Omikron und Ausblick“,05.01.2022,Clemens Wendtner [00:11:32],,0.583333,0.583333,1.245173


## 3. Detect claim sentences

In [14]:
detector = detect_claims.claim_detector(MODEL_NAME, MODEL_WEIGHTS_PATH)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.
2022-02-07 12:59:29.178110: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 12:59:29.188015: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 12:59:29.188681: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 12:59:29.189751: I tensorflow/core/p

In [15]:
doc_table["claim"] = doc_table.apply(lambda x: detector.is_claim(x["sentence"]), axis=1)

In [16]:
doc_table.head()

,sentence,passage_id,title,date,speaker,timestamp,claim
0,Guten Mittag liebe Journalistinnen und Journal...,0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00],False
1,Herzlich willkommen hier zu unserem virtuellen...,0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00],False
2,"Mein Name ist Bastian Zimmermann, und ich bin ...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00],False
3,"Ich freue mich, heute auch unsere drei Experte...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00],False
4,"Man hört immer wieder, wie wichtig Quantentech...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00],True


## 4. Return statements

### 4.1 Highlight claim sentences

In [17]:
relevant_claims = doc_table[(doc_table["claim"]==True)&(doc_table["speaker"]!="Moderator")]

In [18]:
claim_sentences = relevant_claims["sentence"].to_list()

In [19]:
highlight_pdf.highlight_text(claim_sentences, SAMPLE_PDF_PATH, "claims.pdf", color=["green", "yellow"])

Error, sentence not found: "3 Was man auch schon sieht beim Übergang von einem Experiment zu einem richtigen Computer und zu einem Großgerät ist diese ganze Pluralität von Plattformen."
Error, sentence not found: "Das heißt also, Quantenmaschinen bauen, die es in der Größenordnung von vielleicht einigen zehn, mehreren hundert Quantenbits (geben) wird, die noch nicht fehlerkorrigiert sind, aber auf denen man doch schon 4 Quantenrechnungen machen kann."
Error, sentence not found: "Allein in Deutschland gibt es zum Beispiel zwei Exzellenzcluster, einen im Rheinland und einen in 8 Würzburg, die sich damit beschäftigen."
Error, sentence not found: "Wenn man die Ionen und die Supraleiter vergleicht: Die Ionen haben eine unglaublich geringe Fehlerrate und die Supraleiter haben eine etwas höhere Taktfrequenz, die langsamste Operation ist in Nanosekunden gemessen schneller und es kommt dann wirklich auf die Anwendung an, was wie 10 wichtig ist."
Error, sentence not found: "Ein Aspekt der Tatsac

### 4.2 Highlight statements

In [20]:
c = "yellow"
highlight_pdf.highlight_text([], SAMPLE_PDF_PATH, "statements.pdf")

passage_id_old = None
segment_id_old = None

for row in relevant_claims.iterrows():
    passage_id = row[1]["passage_id"]
    segment_id = row[1]["segment_id"]

    if passage_id_old==passage_id and segment_id_old == segment_id:
        continue
    else:
        sentences = doc_table[(doc_table["passage_id"]==passage_id)&(doc_table["segment_id"]==segment_id)]["sentence"].to_list()
        highlight_pdf.highlight_text(sentences, "statements.pdf", "statements.pdf", color=c)
        passage_id_old = passage_id
        segment_id_old = segment_id

        if c == "yellow":
            c = "red"
        else:
            c = "yellow"
        

Error, sentence not found
Error, sentence not found
Error, sentence not found


### 4.3 Highlight statements with claims

In [24]:
c = "yellow"
highlight_pdf.highlight_text([], SAMPLE_PDF_PATH, "statements_claims.pdf")

passage_id_old = None
segment_id_old = None

for row in relevant_claims.iterrows():
    passage_id = row[1]["passage_id"]
    segment_id = row[1]["segment_id"]

    if passage_id_old==passage_id and segment_id_old == segment_id:
        continue
    else:
        sentences = doc_table[(doc_table["passage_id"]==passage_id)&(doc_table["segment_id"]==segment_id)]["sentence"].to_list()
        highlight_pdf.highlight_text(sentences, "statements_claims.pdf", "statements_claims.pdf", color=c)
        passage_id_old = passage_id
        segment_id_old = segment_id

        if c == "yellow":
            c = "red"
        else:
            c = "yellow"
        

Error, sentence not found
Error, sentence not found
Error, sentence not found


In [25]:
highlight_pdf.highlight_text(claim_sentences, "statements_claims.pdf", "statements_claims.pdf", color="green")

Error, sentence not found
Error, sentence not found
